# LangGraph with Multi-Agent Workflow

Multiple agents interacting in Workflow design

In [99]:
import os
import json
import operator
from loguru import logger
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage
)
from langchain.tools import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import FunctionMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from typing import TypedDict, Annotated, List

from langgraph.graph import StateGraph, END

In [2]:
# OpenAI API Key solely to use embedding model
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

In [3]:
# Tools to be called by LLM
tools = [TavilySearchResults(max_results=1)]

tool_executor = ToolExecutor(tools) # Can invoke an action??? - Yes, with the right invocation recipe

In [4]:
# Load LLM
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1)

In [12]:
# There is no additional kwargs or tool call
llm.invoke("who played against Universitario last night?")

AIMessage(content="I'm sorry, but I can't provide real-time or the most recent information, including sports results or schedules, as my last update was in September 2023. To find out who played against Universitario last night, I recommend checking the latest sports news on a reliable sports news website, the official league website, or Universitario's official website or social media channels.", response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 16, 'total_tokens': 93}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e36c9187-c49f-449b-91ce-8c8c2a6effab-0')

In [5]:
# Bind tools to llm
llm_bound = llm.bind_tools(tools)

In [54]:
sample_query = HumanMessage(content="who played against Universitario last night?")
sample_call = llm_bound.invoke([sample_query])
sample_call


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_L4j3A0mT1AzIxbBuB7rKKsDF', 'function': {'arguments': '{"query":"Universitario match result last night"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ad80cf02-9b21-4536-9595-ad65c23161e8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Universitario match result last night'}, 'id': 'call_L4j3A0mT1AzIxbBuB7rKKsDF'}])

In [10]:
print(sample_call.additional_kwargs)
print(sample_call.additional_kwargs.keys())
sample_call.tool_calls

{'tool_calls': [{'id': 'call_f12XooXziw7ecQ2iY75U7CJF', 'function': {'arguments': '{"query":"Universitario match result last night"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}
dict_keys(['tool_calls'])


[{'name': 'tavily_search_results_json',
  'args': {'query': 'Universitario match result last night'},
  'id': 'call_f12XooXziw7ecQ2iY75U7CJF'}]

In [31]:
# json.loads(last_message.additional_kwargs["function_call"]["arguments"])

print(sample_call.additional_kwargs.keys())
print(sample_call.additional_kwargs["tool_calls"])

# json.loads(sample_call.tool_calls[0]["args"]) # ERROR, single quote string like dict cannot be converted to dict

json.loads(sample_call.additional_kwargs["tool_calls"][0]["function"]["arguments"])



dict_keys(['tool_calls'])
[{'id': 'call_f12XooXziw7ecQ2iY75U7CJF', 'function': {'arguments': '{"query":"Universitario match result last night"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]


{'query': 'Universitario match result last night'}

The tool invocation is the recipe needed to be executed

In [35]:
invocation_sample = ToolInvocation(
        tool=sample_call.additional_kwargs["tool_calls"][0]["function"]["name"],
        tool_input=json.loads(sample_call.additional_kwargs["tool_calls"][0]["function"]["arguments"]),
    )

invocation_sample

ToolInvocation(tool='tavily_search_results_json', tool_input={'query': 'Universitario match result last night'})

The executor (made up of the tools themselves) will execute the recipe (invocation)

In [66]:
invocation_execution = tool_executor.invoke(invocation_sample)
invocation_execution

[{'url': 'https://www.sofascore.com/team/football/universitario/2305',
  'content': 'Universitario previous match was against Comerciantes Unidos in Liga 1, the match ended with result 6 - 0 (Universitario won the match). Universitario fixtures tab is showing the last 100 football matches with statistics and win/draw/lose icons. There are also all Universitario scheduled matches that they are going to play in the future.'}]

In [67]:
str(invocation_execution)

"[{'url': 'https://www.sofascore.com/team/football/universitario/2305', 'content': 'Universitario previous match was against Comerciantes Unidos in Liga 1, the match ended with result 6 - 0 (Universitario won the match). Universitario fixtures tab is showing the last 100 football matches with statistics and win/draw/lose icons. There are also all Universitario scheduled matches that they are going to play in the future.'}]"

In [68]:
print(invocation_sample.tool)
print(FunctionMessage(content=str(invocation_execution), 
                name=invocation_sample.tool)
)

print(llm_bound.invoke(
    [
        FunctionMessage(content=str(invocation_execution), 
                        name=invocation_sample.tool)
    ]
))


tavily_search_results_json
content="[{'url': 'https://www.sofascore.com/team/football/universitario/2305', 'content': 'Universitario previous match was against Comerciantes Unidos in Liga 1, the match ended with result 6 - 0 (Universitario won the match). Universitario fixtures tab is showing the last 100 football matches with statistics and win/draw/lose icons. There are also all Universitario scheduled matches that they are going to play in the future.'}]" name='tavily_search_results_json'
content="Universitario's previous match was against Comerciantes Unidos in Liga 1, and Universitario won the match with a score of 6 - 0. For more details on their fixtures and past match statistics, you can visit [SofaScore](https://www.sofascore.com/team/football/universitario/2305)." response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 192, 'total_tokens': 267}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id=

In [69]:
print(isinstance(sample_query, ToolMessage))
print(isinstance(sample_call, ToolMessage))
print(isinstance(FunctionMessage(content=str(invocation_execution), 
                        name=invocation_sample.tool), 
                        ToolMessage))

False
False
False


In [78]:
print(llm_bound.invoke(
    [
        sample_query,
        sample_call,
        FunctionMessage(content=str(invocation_execution), 
                        name=invocation_sample.tool)
    ]
))

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_L4j3A0mT1AzIxbBuB7rKKsDF", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}

### Build Agent Graph

#### Define tools

In [80]:
web_search_tool = TavilySearchResults(max_results=5)

Tool `MUST HAVEs`: name, description and args

In [83]:
print(web_search_tool.name)
print(web_search_tool.description)
print(web_search_tool.args)

tavily_search_results_json
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}


We can also define `Custom tools` with MUST HAVEs

In [90]:
@tool
def addition_tool(x, y):
    """Addition of two numbers
    :param: x: The first number to be added 
    :param: y: The second number to be added"""

    return x+y

@tool
def multiplication_tool(x, y):
    """Multiplication of two numbers
    :param: x: The first number to be multiplied 
    :param: y: The second number to be multiplied"""

    return x*y

In [86]:
print(addition_tool.name)
print(addition_tool.description)
print(addition_tool.args)

addition_tool
addition_tool(x, y) - Addition of two number
    :param: x: The first number to be added 
    :param: y: The second number to be added
{'x': {'title': 'X'}, 'y': {'title': 'Y'}}


In [91]:
tools = [addition_tool, multiplication_tool, web_search_tool]
tool_dict = {i.name:i for i in tools}

tool_dict

{'addition_tool': StructuredTool(name='addition_tool', description='addition_tool(x, y) - Addition of two numbers\n    :param: x: The first number to be added \n    :param: y: The second number to be added', args_schema=<class 'pydantic.main.addition_toolSchema'>, func=<function addition_tool at 0x117c77e50>),
 'multiplication_tool': StructuredTool(name='multiplication_tool', description='multiplication_tool(x, y) - Multiplication of two numbers\n    :param: x: The first number to be multiplied \n    :param: y: The second number to be multiplied', args_schema=<class 'pydantic.main.multiplication_toolSchema'>, func=<function multiplication_tool at 0x117c77670>),
 'tavily_search_results_json': TavilySearchResults()}

#### Define Graph state

In [ ]:
class StrategyAgentState(TypedDict):
    user_query: str
    steps: Annotated[List, operator.add] # List that holds steps identified by planner node that need to be executed by the tool execution node
    step_no: int
    results: dict
    final_response: str
    end:bool

#### Define nodes

`Planner node` (Manager). A couple of points:
- Notice how we use the `JsonOutputToolsParser` to generate an invocation in JSON format, that is, the arguments of the functions call as JSON (more [here](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/openai_tools/))

In [93]:
def planner_node(state):

    """
    Plans steps and directs agents
    """

    user_question = state["user_query"]
    steps = state["steps"]
    results = state["results"]
    end = state["end"]

    # If we are at the beginning of multi-agent conversation and no results have been produced yet
    if results is None:
        system_prompt = """
        You are a helpful assistant who is good a mathematics and can search the internet.
        Do not execute yourself, let the tools perform operations. Call one tool at a time.
        """

        prompt_template = ChatPromptTemplate(
            [
                ("system", system_prompt),
                ("user", "{user_question}")
            ]
        )

        llm_planner = prompt_template | llm.bind_tools(tools) | JsonOutputToolsParser()

        # Get invocation (recipe of tool steps)
        steps = llm_planner.invoke({"user_question":user_question})

        # Log steps
        logger.info(f"LLM Generated plan: {steps}")

        # We are returning, the recipe in JSON, that is, the list of steps that need to be executed
        return {"steps":steps}

#### Define Graph Structure